In [1]:
import pandas as pd
from  ETL_functions import *

In [2]:
#leamos cada uno de los archivos y transformemoslos a todos a .csv

df_json = pd.read_json('/home/gero/Documentos/HENRY/LAB I/PI01_DATA_ENGINEERING/Datasets/precios_semana_20200503.json')
df_txt = pd.read_csv('/home/gero/Documentos/HENRY/LAB I/PI01_DATA_ENGINEERING/Datasets/precios_semana_20200518.txt',delimiter='|')
df_csv = pd.read_csv('/home/gero/Documentos/HENRY/LAB I/PI01_DATA_ENGINEERING/Datasets/precios_semana_20200413.csv',encoding='utf_16')

df_products_parquet = pd.read_parquet('/home/gero/Documentos/HENRY/LAB I/PI01_DATA_ENGINEERING/Datasets/producto.parquet',engine='pyarrow')
df_sucursales = pd.read_csv('/home/gero/Documentos/HENRY/LAB I/PI01_DATA_ENGINEERING/Datasets/sucursal.csv')

In [3]:
#hojas del excel. 
hoja1 = 'precios_20200426_20200426'
hoja2 = 'precios_20200419_20200419'
df_excel_semana_426 = pd.read_excel('/home/gero/Documentos/HENRY/LAB I/PI01_DATA_ENGINEERING/Datasets/precios_semanas_20200419_20200426.xlsx',sheet_name=hoja1)


In [4]:
df_excel_semana_419 = pd.read_excel('/home/gero/Documentos/HENRY/LAB I/PI01_DATA_ENGINEERING/Datasets/precios_semanas_20200419_20200426.xlsx',sheet_name=hoja2)

In [5]:
#apliquemos nuestras funciones ETL para cada dataframe. 
df_products_parquet = etl_productos(df_products_parquet)

In [6]:
df_sucursales = elt_sucursal(df_sucursales)

In [7]:
df_csv = etl_precios(df_csv)

In [8]:
df_txt = etl_precios(df_txt)

In [ ]:
#a partir de aqui veremos que ademas de aplicarle la respectiva funcion ETL a cada dataframe, se le haran algunas otras modificaciones especificas para ese df. 

In [9]:
df_json['producto_id'] = df_json['producto_id'].astype(str)# nos convierte la columna producto_id a tipo string

In [10]:
df_json['producto_id'] = [i.lstrip('0') for i in df_json['producto_id']] #nos saca los ceros a al izquierda de la columna producto_id

In [11]:
df_json = etl_precios(df_json)

In [12]:
df_excel_semana_419.sucursal_id = df_excel_semana_419.sucursal_id.apply(lambda _ : str(_)) #convertimos a string

In [13]:
df_excel_semana_419['sucursal_id'] = df_excel_semana_419['sucursal_id'].apply(lambda x: x.split(' ')[0]) #nos saca la hora.

In [14]:
df_excel_semana_419['producto_id'] = df_excel_semana_419['producto_id'].astype(str).str.split('.',expand=True)[0]

In [15]:
df_excel_semana_419 = etl_precios(df_excel_semana_419)

In [16]:
df_excel_semana_426['producto_id'] = df_excel_semana_426['producto_id'].astype(str).str.split('.',expand=True)[0]

In [17]:
df_excel_semana_426 = etl_precios(df_excel_semana_426) 

producto_id ya esta limpio


In [ ]:
# Terminada la limpieza de los datos, lo que haremos ahora sera unir las tablas que contienen los precios, los id_sucursales y los id_producto. 
# Vemos aqui abajo que usamos el metodo concat para unir todos los dataframes correspondientes. 

In [18]:
precios = pd.concat([df_json,df_txt,df_csv,df_excel_semana_426,df_excel_semana_419],axis=0)
precios.shape[0] #la tabla precios contiene aproximadamente 2 millones de registros. 

2217125

In [ ]:
#ya terminado todo el proceso de transformacion, empezemos nuestra carga a la base de datos mysql. 

In [20]:
from sql import *
#importamos el script de la conexion a mysql.

In [21]:
#conectamos los 3 df que nos quedaron a mysql. 
precios.to_sql('precios',conexion,index=False)

In [ ]:
df_sucursales.to_sql('sucursales',conexion,index=False)

In [ ]:
df_products_parquet.to_sql('productos',conexion,index=False)